# ⚙️ Módulo 9 — Concurrencia con Threads

En este notebook aprenderás:

- Qué es el **GIL** (Global Interpreter Lock)
- Cuándo usar *threads* en Python
- Crear hilos con `threading.Thread`
- Problemas reales: **race conditions**
- Usar `Lock` para evitar condiciones de carrera
- Buenas prácticas en concurrencia

---

# 1. Que es el GIL

El GIL (Global Interpreter Lock) es un bloqueo global que impide que dos threads ejecuten bytecode de Python al mismo tiempo.

**Conclusion:** los hilos NO aceleran tareas CPU-bound.
**Pero SI mejoran tareas IO-bound:** lectura/escritura, red, ficheros, esperas.

---

# 2️⃣ Crear un thread básico


In [ ]:
import threading
import time

def tarea():
    print('Ejecutando tarea en thread...')
    time.sleep(1)
    print('Tarea completada')

hilo = threading.Thread(target=tarea)
hilo.start()
hilo.join()
print('Fin del programa principal')

---
# 3️⃣ Múltiples threads
Crear varios threads simultáneos:

In [ ]:
def tarea_num(i):
    print(f'Thread {i} iniciando...')
    time.sleep(0.3)
    print(f'Thread {i} terminado')

hilos = []
for i in range(5):
    h = threading.Thread(target=tarea_num, args=(i,))
    h.start()
    hilos.append(h)

for h in hilos:
    h.join()

---
# 4️⃣ Race Condition (condición de carrera)

Un ejemplo clásico: varios hilos modifican la misma variable **sin sincronización**.

In [ ]:
contador = 0

def incrementar():
    global contador
    for _ in range(100000):
        contador += 1

h1 = threading.Thread(target=incrementar)
h2 = threading.Thread(target=incrementar)

h1.start(); h2.start()
h1.join(); h2.join()

contador

🔍 **El resultado debería ser 200000**, pero rara vez lo es → hay una condición de carrera.

---
# 5️⃣ Solución: usar un `Lock`


In [ ]:
contador = 0
lock = threading.Lock()

def incrementar_con_lock():
    global contador
    for _ in range(100000):
        with lock:
            contador += 1

h1 = threading.Thread(target=incrementar_con_lock)
h2 = threading.Thread(target=incrementar_con_lock)
h1.start(); h2.start()
h1.join(); h2.join()

contador

---
# 6️⃣ Buenas prácticas con threads

- Usar `Thread` solo para IO-bound
- Proteger recursos compartidos con `Lock`
- Evitar demasiados threads → usar `ThreadPoolExecutor`
- Siempre usar `.join()` para sincronizar
- No mezclar threads y multiprocessing sin necesidad

---

# 7️⃣ Ejercicio práctico

### 🧩 Objetivos
1. Crear 10 threads
2. Cada uno deberá:
   - Dormir entre 0.1 y 0.5 segundos (aleatorio)
   - Devolver un valor
3. Guardar los resultados en una lista compartida (usar `Lock`)
4. Mostrar los resultados ordenados

Escribe tu código abajo:

In [ ]:
# TU CÓDIGO AQUÍ


---
# ✅ Soluciones (ocultas)

<details>
<summary>Mostrar solución</summary>

```python
import random

resultados = []
lock = threading.Lock()

def trabajo(i):
    tiempo = random.uniform(0.1, 0.5)
    time.sleep(tiempo)
    with lock:
        resultados.append((i, tiempo))

hilos = []
for i in range(10):
    h = threading.Thread(target=trabajo, args=(i,))
    h.start()
    hilos.append(h)

for h in hilos:
    h.join()

sorted(resultados, key=lambda x: x[1])
```
</details>